# 🏗️ Chapter 10: 3D Surface Reconstruction

Turning point clouds into watertight meshes is a critical step for visualization, 3D printing, and physics simulations. In this chapter, we cover:

1.  **Ball Pivoting Algorithm (BPA)**: Rolling a ball over points to connect them.
2.  **Poisson Surface Reconstruction**: Solving a mathematical equation (Poisson) to fit a smooth surface.

**Objectives:**
*   Reconstruct a mesh from the raw point cloud.
*   Compare the quality of BPA vs. Poisson.

In [ ]:
import numpy as np
import open3d as o3d
import matplotlib.pyplot as plt

## 1. Load Point Cloud

We need a point cloud with **Normals**. Meshing algorithms rely heavily on normal vectors to know "which way is out".

In [ ]:
pcd = o3d.io.read_point_cloud("../DATA/sample_w_normals.xyz")

# If normals are missing, compute them:
if not pcd.has_normals():
    print("Estimating normals...")
    pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.1, max_nn=30))
    # Orient normals consistently towards camera location (0,0,0) or similar
    pcd.orient_normals_consistent_tangent_plane(10)

o3d.visualization.draw_geometries([pcd], window_name="Input Point Cloud", point_show_normal=True)

## 2. Ball Pivoting Algorithm (BPA)

BPA is intuitive: imagine dropping a ball on the points. Where it touches 3 points without falling through, a triangle is formed.

**Pros:** Preserves original points exactly. Good for dense, even data.
**Cons:** Leaves holes if point density varies or "ball size" is wrong.

In [ ]:
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist
print(f"Using ball radius: {radius:.4f}")

# Create mesh
mesh_bpa = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(
    pcd, o3d.utility.DoubleVector([radius, radius * 2])
)

# Visualize
mesh_bpa.compute_vertex_normals()
mesh_bpa.paint_uniform_color([0.5, 0.5, 0.5])
o3d.visualization.draw_geometries([mesh_bpa], window_name="BPA Mesh")

## 3. Poisson Surface Reconstruction

Poisson fits an implicit function (water-tight surface) to the oriented point normals. It's robust to noise and varying density.

**Pros:** Creates watertight, smooth models. Handles noise well.
**Cons:** Can "over-smooth" details. Creates a "blob" around the object (requires cropping).

In [ ]:
print("Running Poisson reconstruction...")
mesh_poisson, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(
    pcd, depth=9, width=0, scale=1.1, linear_fit=False
)

# Poisson creates a bounding box mesh (often excess geometry). We can filter by density.
print("Visualizing raw Poisson mesh (may have artifacts)...")
o3d.visualization.draw_geometries([mesh_poisson], window_name="Raw Poisson Mesh")

## 4. Cleaning the Mesh

Poisson often creates "ghost" geometry in areas with no points. We remove vertices with low density support.

In [ ]:
# Crop based on density (low density = hallucinated parts)
densities = np.asarray(densities)
density_threshold = np.percentile(densities, 5)
vertices_to_remove = densities < density_threshold
mesh_poisson.remove_vertices_by_mask(vertices_to_remove)

mesh_poisson.compute_vertex_normals()
mesh_poisson.paint_uniform_color([0.5, 0.5, 0.5])
o3d.visualization.draw_geometries([mesh_poisson, pcd], window_name="Cleaned Poisson Mesh")

5. Exporting

Save your mesh for use in Blender, Unity, or 3D printing.

In [ ]:
o3d.io.write_triangle_mesh("../RESULTS/reconstructed_bpa.ply", mesh_bpa)
o3d.io.write_triangle_mesh("../RESULTS/reconstructed_poisson.ply", mesh_poisson)